In [9]:
!pip install xmltodict

You should consider upgrading via the '/home/A19893678/local/miniconda3/bin/python -m pip install --upgrade pip' command.


In [61]:
import xmltodict
import os
def parse_file(path,file=None):
    path=path+file
    def parse_object(obj):
        objdict={}
        objdict = {k:v for k,v in obj['bndbox'].items()}
        #print(objdict)
        objdict['label']=obj['name']
        outdict = {**objdict,**metadata}
        return outdict
    #print(path)
    with open(path,'r') as f:
        parsed=xmltodict.parse(f.read())
        annot=parsed['annotation']
        parselist=[]
        metadata={}
        metadata['file']=annot['filename']
        size={k:v for k,v in annot['size'].items()}
        
        metadata={**metadata,**size}
        #print('Metadata',metadata)
        if type(annot['object']) is list:
            for obj in annot['object']:
                parselist.append(parse_object(obj))
        else:
            parselist.append(parse_object(annot['object']))
        return parselist
parse_file(os.getcwd()+'/','test.xml')


/home/A19893678/GDA3B2021/ProjectFaceMask/test.xml


[{'xmin': '79',
  'ymin': '105',
  'xmax': '109',
  'ymax': '142',
  'label': 'without_mask',
  'file': 'maksssksksss0.png',
  'width': '512',
  'height': '366',
  'depth': '3'},
 {'xmin': '185',
  'ymin': '100',
  'xmax': '226',
  'ymax': '144',
  'label': 'with_mask',
  'file': 'maksssksksss0.png',
  'width': '512',
  'height': '366',
  'depth': '3'},
 {'xmin': '325',
  'ymin': '90',
  'xmax': '360',
  'ymax': '141',
  'label': 'without_mask',
  'file': 'maksssksksss0.png',
  'width': '512',
  'height': '366',
  'depth': '3'}]

In [ ]:
import os
import pandas as pd
datalist=[]
for file in os.listdir('annotations'):
    objlist=parse_file('./annotations/',file)
    datalist.extend(objlist)
annotdf=pd.DataFrame(datalist)


In [64]:
annotdf

,xmin,ymin,xmax,ymax,label,file,width,height,depth
0,76,236,149,349,without_mask,maksssksksss530.png,301,400,3
1,241,60,271,97,with_mask,maksssksksss367.png,400,267,3
2,89,169,215,309,without_mask,maksssksksss442.png,301,400,3
3,156,80,222,155,with_mask,maksssksksss242.png,400,300,3
4,75,75,158,161,with_mask,maksssksksss215.png,400,400,3
...,...,...,...,...,...,...,...,...,...
4067,235,43,272,87,with_mask,maksssksksss116.png,400,225,3
4068,304,68,336,102,with_mask,maksssksksss116.png,400,225,3
4069,379,61,399,96,with_mask,maksssksksss116.png,400,225,3
4070,89,210,180,303,with_mask,maksssksksss339.png,400,400,3
